<a href="https://colab.research.google.com/github/laisab/IC/blob/main/rascunho_ic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate

In [3]:
# Dicionário com datasets e seus respectivos targets
dicio_datasets = {
    'Adult.csv': 'income',
    'Biomed.csv': 'class',
    'COMPAS.csv': 'two_year_recid',
    'ContraceptiveMethodChoice.csv': "'Contraceptive_method_used'",
    'DefaultOfCreditCardClients.csv': 'default.payment.next.month',
    'GermanCreditRisk.csv': "class'",
    'HabermansSurvivalData.csv': "'Survival_status'",
    'HeartDiseaseStatlog.csv': 'target',
    'IndianLiverPatientDataset.csv': 'Class',
    'IrishEducationalTransitionsData.csv': 'Leaving_Certificate',
    'LowBirthWeightData.csv': 'binaryClass',
    'PimaIndiansDiabetesDatabase.csv': "'class'",
    'PopularKids.csv': 'Goals',
    'PortugueseBankMarketing.csv': 'Subscription',
    'Schizo.csv': 'class',
    'SocialMobility.csv': 'binaryClass',
    'ThyroidSicknessDetermination.csv': 'Class'
}

**Idade**
*   Menor de idade: < 18
*   Jovem: 18 aos 30 anos
*   Adulto: 31 aos 60 anos
*   Idoso: acima dos 60 anos


**Idade menor** (dataset PopularKids)
*  Criança: <= 9 anos
*  Pré-adolescente: 10 aos 13 anos
*  Adolescente: > 13 anos

**Educação** (dataset DefaultOfCreditCardClients)
*   Pós-graduação: 1
*   Universidade: 2
*   Ensino médio: 3
*   Outros: 4


**Estado civil** (dataset DefaultOfCreditCardClients)
*   Casado: 1
*   Solteiro: 2
*   Outros: 3
*   Desconhecido: 0

**Religião** (dataset ContraceptiveMethodChoice)
*   Outras: 0
*   Islamismo: 1

**Esposa trabalha** (dataset ContraceptiveMethodChoice)
*   Trabalha: 0
*   Não trabalha: 1

**Padrão de vida e nível de educação** (dataset ContraceptiveMethodChoice)
*   Baixo: 1
*   Médio-baixo: 2
*   Médio-alto: 3
*   Alto: 4

**Raça** (dataset LowBirthWeightData)
*   Branco: 1
*   Preto: 2
*   Outras: 3

**Gênero**

Para os datasets COMPAS e HeartDiseaseStatlog
*   Mulher: 0
*   Homem: 1

Para o dataset DefaultOfCreditCardClients
*   Homem: 1
*   Mulher: 2

In [4]:
idade = {
    'bins': [14, 18, 30, 60, np.inf],
    'labels': ['menor de idade', 'jovem', 'adulto', 'idoso']
}

idade_menor = {
    'bins': [0, 9, 13, np.inf],
    'labels': ['criança', 'pre-adolescente', 'adolescente']
}

educacao = {
    1: 'pós-graduação',
    2: 'universidade',
    3: 'ensino médio',
    4: 'outros'
}

estado_civil = {
    1: 'casado',
    2: 'solteiro',
    3: 'outros',
    0: 'desconhecido'
}

religiao = {
    0: 'outras',
    1: 'islamismo'
}

esposa_trabalha = {
    0: 'trabalha',
    1: 'não trabalha'
}

padrao_vida_educacao = {
    1: 'baixo',
    2: 'médio-baixo',
    3: 'médio-alto',
    4: 'alto'
}

raca = {
    1: 'branco',
    2: 'preto',
    3: 'outros'
}

genero_1 = {
    0: 'female',
    1: 'male'
}

genero_2 = {
    1: 'male',
    2: 'female'
}

In [5]:
# Dicionário com datasets e seus respectivos atributos sensíveis
dicio_atributos_sensiveis = {
    'Adult.csv': {
        'age': idade,
        'education': None,
        'marital.status': None,
        'relationship': None,
        'race': None,
        'sex': None,
        'native.country': None
    },

    'Biomed.csv': {
        'Age_of_patient': idade
    },

    'COMPAS.csv': {
      'sex': genero_1,
      'age': idade
    },

    'ContraceptiveMethodChoice.csv': {
      "'Wifes_age'": idade,
      "'Wifes_education'": padrao_vida_educacao,
      "'Husbands_education'": padrao_vida_educacao,
      "'Wifes_religion'": religiao,
      "'Wifes_now_working%3F'": esposa_trabalha,
      "'Husbands_occupation'": None,
      "'Standard-of-living_index'": padrao_vida_educacao
    },

    'DefaultOfCreditCardClients.csv': {
      'SEX': genero_2,
      'EDUCATION': educacao,
      'MARRIAGE': estado_civil,
      'AGE': idade
    },

    'GermanCreditRisk.csv': {
      "personal_status'": None,
      "age'": idade,
      "housing'": None,
      "job'": None
    },

    'HabermansSurvivalData.csv': {
      "'Age_of_patient_at_time_of_operation'": idade
    },

    'HeartDiseaseStatlog.csv': {
      'age': idade,
      'sex': genero_1
    },

    'IndianLiverPatientDataset.csv': {
      'V1': idade,
      'V2': None
    },

    'IrishEducationalTransitionsData.csv': {
      'Sex': None,
      'Prestige_score': None
    },

    'LowBirthWeightData.csv': {
      'AGE': idade,
      'RACE': raca,
    },

    'PimaIndiansDiabetesDatabase.csv': {
      "'age'": idade,
    },

    'PopularKids.csv': {
      'Gender': None,
      'Age': idade_menor,
      'Race': None,
      'Urban/Rural': None
    },

    'PortugueseBankMarketing.csv': {
      'Age': idade,
      'Job': None,
      'Marital Status': None,
      'Education': None
    },

    'Schizo.csv': {
      'sex': None
    },

    'SocialMobility.csv': {
      'family_structure': None,
      'race': None
    },

    'ThyroidSicknessDetermination.csv': {
      'age': None,
      'sex': None
    }
}

In [6]:
caminho_drive = '/content/drive/MyDrive/IC/Datasets/'
novo_caminho = '/content/drive/MyDrive/Teste/'
caminho_resultados = '/content/drive/MyDrive/Teste/Testando/'

In [7]:
# Formatação dos dados
def formatar_dados(resultados_para_formatacao):
  formatado = {}

  for chave, valor in resultados_para_formatacao.items():
    if isinstance(valor, (int, float)):
      formatado[chave] = f'{valor:.3f}' if not pd.isna(valor) else "NaN"
    else:
      formatado[chave] = valor

  return formatado

In [8]:
# Separação das features e target
def separar_feature_target(caminho_completo):

  try:
    df = pd.read_csv(caminho_completo)

    # Obtém o nome do target
    nome_arquivo = os.path.basename(caminho_completo)
    nome_target = dicio_datasets.get(nome_arquivo)

    if not nome_target:
      raise ValueError(f'Coluna target não encontrada para o arquivo {nome_arquivo}')

    # Separa features (X) e target (y)
    X = df.drop(columns=[nome_target])
    y = df[nome_target]

    # Dicionário para armazenar o mapeamento de colunas OneHotEncoder
    # Exemplo: {'sex': ['sex_female', 'sex_male']}
    colunas_onehot = {}

    # Codificação de features categóricas com OneHotEncoder
    colunas_categoricas = X.select_dtypes(include=['object']).columns

    # Verificação de colunas categóricas para evitar erros com datasets numéricos
    if colunas_categoricas.size > 0:
      # Lida com categorias desconhecidas em dados futuros
      enc = OneHotEncoder(handle_unknown='ignore')
      X_encoded = enc.fit_transform(X[colunas_categoricas]).toarray()

      # Nomes das novas features criadas pelo OneHotEncoder
      novo_nome_X = enc.get_feature_names_out(colunas_categoricas)
      X_encoded_df = pd.DataFrame(X_encoded, columns=novo_nome_X)

      # Para cada coluna categórica original, a lista das novas colunas são armazenadas
      for coluna_original in colunas_categoricas:
        # Dicionário "colunas_onehot" => coluna_original é a chave e a lista é o valor
        colunas_onehot[coluna_original] = [nome for nome in novo_nome_X if nome.startswith(f'{coluna_original}_')]

      # Remove as colunas categóricas originais
      X = X.select_dtypes(exclude=['object'])
      X = pd.concat([X, X_encoded_df], axis=1)

    return X, y, colunas_onehot

  except Exception as e:
    print(f'\nErro ao separar os dados de {nome_arquivo}: {e}')
    return None

In [9]:
# Predição dos datasets
def predizer_dataset(modelo, X, y, cv):
  try:
    y_pred = cross_val_predict(modelo, X, y, cv=cv, method='predict')
    y_prob = cross_val_predict(modelo, X, y, cv=cv, method='predict_proba')
    return y_pred, y_prob

  except Exception as e:
    print(f'\nErro ao predizer dataset: {e}')
    return None

In [10]:
# Avaliação dos datasets
def avaliar_dataset(y, y_pred, y_prob):
  try:
    metricas = {
      'accuracy': accuracy_score,
      'f1_score': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro', zero_division=0),
      'roc_auc_score': lambda y_true, y_pred_proba: roc_auc_score(y_true, y_pred_proba, average='macro', multi_class='ovr') if len(np.unique(y_true)) > 2 else (roc_auc_score(y_true, y_pred_proba[:, 1]) if len(np.unique(y_true)) == 2 else np.nan)
    }

    avaliacao = {}
    for metrica, metrica_funcao in metricas.items():
      try:
        if metrica == 'roc_auc_score':
          # AUC não é calculada para target com menos de 2 classes únicas
          if len(np.unique(y)) < 2:
            avaliacao[metrica] = np.nan
          else:
            avaliacao[metrica] = metrica_funcao(y, y_prob)
        else:
          avaliacao[metrica] = metrica_funcao(y, y_pred)
      except Exception as e:
        print(f'\nErro ao calcular {metrica}: {e}')
        avaliacao[metrica] = None

    return avaliacao

  except Exception as e:
    print(f'\nErro ao avaliar dataset: {e}')
    return None

In [11]:
# Avaliação dos segmentos
def avaliar_segmentos(X, y, y_pred, y_prob, X_onehot, avaliar_dataset_func, auc_geral, dataset="", atr_sensivel=None, num_bins=5):
  """
  Args:
    X: features
    y: target
    y_pred: predições
    y_prob: probabilidades
    X_onehot: colunas OneHotEncoder
    avaliar_dataset_func: função avaliar_dataset
    dataset: nome do dataset
    atributos_sensiveis: dicionário com atributos sensíveis
    bins: número de segmentos para atributos numéricos sem label (padrão = 5)
  """

  res_segmentos = []

  for coluna_original, bin_labels in atr_sensivel.items():
    if coluna_original not in X.columns and coluna_original not in X_onehot:
      print(f'  Atributo {coluna_original} não encontrado para o dataset {dataset}')
      continue

    # print(f'\n  Avaliação dos segmentos para o atributo {coluna_original} do dataset {dataset}')

    # Atributos transformados pelo OneHotEncoder
    if coluna_original in X_onehot:
      colunas_onehot = X_onehot[coluna_original] # Lista das colunas geradas pelo OneHot

      for coluna_onehot in colunas_onehot:
        idx = X[coluna_onehot] == 1

        if idx.sum() > 0:
          # Subconjuntos para o segmento atual
          subset_y = y[idx]
          subset_y_pred = y_pred[idx]
          subset_y_prob = y_prob[idx]

          # Extrai o nome original da categoria
          nome_categorico = coluna_onehot.replace(f'{coluna_original}_', '')

          # print(f'  Segmento {nome_categorico}')
          avaliacao = avaliar_dataset_func(subset_y, subset_y_pred, subset_y_prob)
          avaliacao_formatada = formatar_dados(avaliacao)
          # print(avaliacao_formatada)

          auc_segmento = avaliacao.get('roc_auc_score')
          diferenca_auc = np.nan

          auc_geral_formatada = f'{auc_geral:.3f}' if not pd.isna(auc_geral) else "NaN"

          if auc_segmento is not None and auc_geral is not None and not np.isnan(auc_segmento) and not np.isnan(auc_geral) and auc_geral != 0:
            diferenca_auc = ((auc_geral - auc_segmento) / auc_geral) * 100

          diferenca_auc_formatada = f'{diferenca_auc:.2f}' if not pd.isna(diferenca_auc) else "NaN"

          # Número de instância do segmento
          n_instancias = idx.sum()
          n_instancias_porcentagem = f'{n_instancias / len(y) * 100:.3f}'

          res_segmentos.append([coluna_original, nome_categorico, avaliacao_formatada.get('accuracy'), avaliacao_formatada.get('f1_score'), avaliacao_formatada.get('roc_auc_score'), auc_geral_formatada, diferenca_auc_formatada, n_instancias, n_instancias_porcentagem])
        else:
          nome_categorico = coluna_onehot.replace(f'{coluna_original}_', '')
          print(f'  Segmento {nome_categorico} sem dados (com OneHot)')
          res_segmentos.append([coluna_original, nome_categorico, None, None, None, None, None])

    # Atributos sem OneHotEncoder (numéricos ou categóricos que não passaram pelo OneHot)
    else:
      # Obtém a coluna do dataset que corresponde ao atributo sensível
      coluna_dataset = X[coluna_original]
      # Inicializa com os valores da coluna original
      bin_col = coluna_dataset

      classificacao = atr_sensivel.get(coluna_original)

      # Valores de classificação como um dicionário
      if isinstance(classificacao, dict) and 'bins' not in classificacao:
        bin_col = coluna_dataset.map(classificacao).fillna(coluna_dataset)

      # Bins configurados
      elif isinstance(classificacao, dict) and 'bins' in classificacao:
        try:
          bin_col = pd.cut(coluna_dataset, bins=bin_labels['bins'], labels=bin_labels['labels'], include_lowest=True, right=True)
        except Exception as e:
          print(f'  Erro ao configurar os bins para o atributo {coluna_original}: {e}')
          # Mantém o bin_col como a coluna original
          bin_col = coluna_dataset
          continue

      # Bins automáticos
      elif classificacao is None and pd.api.types.is_numeric_dtype(coluna_dataset) and len(coluna_dataset.unique()) > num_bins * 2:
        try:
          bin_col = pd.qcut(coluna_dataset, q=num_bins, duplicates='drop')
        except Exception as e:
          print(f'  Erro ao configurar os bins para o atributo {coluna_original}: {e}')
          # Mantém o bin_col como a coluna original
          bin_col = coluna_dataset
          continue

      # Valores únicos da coluna (original ou após bins de classificação)
      valor_unico = bin_col.unique().tolist()

      # Ordenação dos valores únicos para os intervalos
      try:
        valor_unico.sort(key=lambda x: x.left if isinstance(x, pd.Interval) else x)
      except TypeError:
        # Tentativa de ordenação por conversão para string
        try:
          valor_unico.sort(key=str)
        except TypeError:
          pass

      for valor in valor_unico:
        idx = bin_col == valor

        if idx.sum() > 0:
          # Subconjuntos para o segmento atual
          subset_y = y[idx]
          subset_y_pred = y_pred[idx]
          subset_y_prob = y_prob[idx]

          # Definição do nome do grupo como a string do valor ou do bin
          nome_grupo = str(valor)

          # print(f'  Segmento {nome_grupo}')
          avaliacao = avaliar_dataset_func(subset_y, subset_y_pred, subset_y_prob)
          avaliacao_formatada = formatar_dados(avaliacao)
          # print(avaliacao_formatada)

          auc_segmento = avaliacao.get('roc_auc_score')
          diferenca_auc = np.nan

          auc_geral_formatada = f'{auc_geral:.3f}' if not pd.isna(auc_geral) else "NaN"

          if auc_segmento is not None and auc_geral is not None and not np.isnan(auc_segmento) and not np.isnan(auc_geral) and auc_geral != 0:
            diferenca_auc = ((auc_geral - auc_segmento) / auc_geral) * 100

          diferenca_auc_formatada = f'{diferenca_auc:.2f}' if not pd.isna(diferenca_auc) else "NaN"

          # Número de instância do segmento
          n_instancias = idx.sum()
          n_instancias_porcentagem = f'{n_instancias / len(y) * 100:.3f}'

          res_segmentos.append([coluna_original, nome_grupo, avaliacao_formatada.get('accuracy'), avaliacao_formatada.get('f1_score'), avaliacao_formatada.get('roc_auc_score'), auc_geral_formatada, diferenca_auc_formatada, n_instancias, n_instancias_porcentagem])
        else:
          nome_grupo = str(valor)
          print(f'  Segmento {nome_grupo} sem dados (sem One Hot)')
          res_segmentos.append([coluna_original, nome_grupo, None, None, None, None, None])

  return pd.DataFrame(res_segmentos, columns=['ATRIBUTO', 'SEGMENTO', 'ACURÁCIA', 'F1', 'AUC SEGMENTO', 'AUC DATASET', 'DIFERENÇA AUC (%)', 'NÚMERO DE INSTÂNCIAS', 'NÚMERO DE INSTÂNCIAS (%)'])

In [12]:
# Lista os arquivos csv na pasta
arquivos = [f for f in os.listdir(caminho_drive) if f.endswith('.csv') and f in dicio_datasets]
arquivos

['IndianLiverPatientDataset.csv',
 'HabermansSurvivalData.csv',
 'GermanCreditRisk.csv',
 'PortugueseBankMarketing.csv',
 'DefaultOfCreditCardClients.csv',
 'Adult.csv',
 'COMPAS.csv',
 'ContraceptiveMethodChoice.csv',
 'ThyroidSicknessDetermination.csv',
 'HeartDiseaseStatlog.csv',
 'SocialMobility.csv',
 'LowBirthWeightData.csv',
 'PopularKids.csv',
 'IrishEducationalTransitionsData.csv',
 'Biomed.csv',
 'Schizo.csv',
 'PimaIndiansDiabetesDatabase.csv']

In [20]:
# Processamento de cada dataset - RandomForest
resultados_dataset = []
res_metrica_tabela = []

modelo = RandomForestClassifier(n_estimators=20, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo)

  if dados is not None:
    X, y, X_onehot = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados_dataset = avaliar_dataset(y, predicao, probabilidade)
    resultados_dataset_formatados = formatar_dados(resultados_dataset)
    # print(f'\n  Resultados do dataset {arquivo}: {resultados_dataset_formatados}')

    res_metrica_tabela.append([
      arquivo,
      str(modelo).split('(')[0],
      resultados_dataset_formatados.get('accuracy'),
      resultados_dataset_formatados.get('f1_score'),
      resultados_dataset_formatados.get('roc_auc_score')
    ])

    auc_geral = resultados_dataset.get('roc_auc_score')

    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)

    df_segmentos = avaliar_segmentos(X, y, predicao, probabilidade, X_onehot, avaliar_dataset, auc_geral, arquivo, dicio_atributos_sensiveis.get(arquivo, {}), 5)

    if not df_segmentos.empty:
      df_segmentos['num_dif_auc'] = pd.to_numeric(
          df_segmentos['DIFERENÇA AUC (%)'],
          errors='coerce'
      )

      df_segmentos = df_segmentos.sort_values(
          by='num_dif_auc',
          ascending=False,
          na_position='last'
      )

      df_segmentos = df_segmentos.drop(columns=['num_dif_auc'])

    nome_dataset = arquivo.replace('.csv', '')
    arquivo_segmento = f"Segmentos_{nome_dataset}_RF.csv"
    caminho_segmento = os.path.join(caminho_resultados + "RandomForest/", arquivo_segmento)
    df_segmentos.to_csv(caminho_segmento, index=False)
  else:
    print(f'Não foi possível processar o dataset {arquivo}')

  Segmento nan sem dados (sem One Hot)


In [14]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═════════════════════════════════════╤════════════════════════╤════════════╤═══════╤═══════╕
│ NOME DO DATASET                     │ MODELO                 │   ACURÁCIA │    F1 │   AUC │
╞═════════════════════════════════════╪════════════════════════╪════════════╪═══════╪═══════╡
│ IndianLiverPatientDataset.csv       │ RandomForestClassifier │      0.707 │ 0.576 │ 0.745 │
├─────────────────────────────────────┼────────────────────────┼────────────┼───────┼───────┤
│ HabermansSurvivalData.csv           │ RandomForestClassifier │      0.699 │ 0.565 │ 0.621 │
├─────────────────────────────────────┼────────────────────────┼────────────┼───────┼───────┤
│ GermanCreditRisk.csv                │ RandomForestClassifier │      0.744 │ 0.66  │ 0.76  │
├─────────────────────────────────────┼────────────────────────┼────────────┼───────┼───────┤
│ PortugueseBankMarketing.csv         │ RandomForestClassifier │      0.903 │ 0.712 │ 0.913 │
├─────────────────────────────────────┼─────────────────────

In [15]:
# Processamento de cada dataset - KNN
resultados_dataset = []
res_metrica_tabela = []

modelo = KNeighborsClassifier(n_neighbors=5)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo)

  if dados is not None:
    X, y, X_onehot = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados_dataset = avaliar_dataset(y, predicao, probabilidade)
    resultados_dataset_formatados = formatar_dados(resultados_dataset)
    # print(f'\n  Resultados do dataset {arquivo}: {resultados_dataset_formatados}')

    res_metrica_tabela.append([
      arquivo,
      str(modelo).split('(')[0],
      resultados_dataset_formatados.get('accuracy'),
      resultados_dataset_formatados.get('f1_score'),
      resultados_dataset_formatados.get('roc_auc_score')
    ])

    auc_geral = resultados_dataset.get('roc_auc_score')

    """
    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)
    """

    df_segmentos = avaliar_segmentos(X, y, predicao, probabilidade, X_onehot, avaliar_dataset, auc_geral, arquivo, dicio_atributos_sensiveis.get(arquivo, {}), 5)

    nome_dataset = arquivo.replace('.csv', '')
    arquivo_segmento = f"Segmentos_{nome_dataset}_KNN.csv"
    caminho_segmento = os.path.join(caminho_resultados + "KNN/", arquivo_segmento)
    df_segmentos.to_csv(caminho_segmento, index=False)
  else:
    print(f'Não foi possível processar o dataset {arquivo}')

  Segmento nan sem dados (sem One Hot)


In [16]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═════════════════════════════════════╤══════════════════════╤════════════╤═══════╤═══════╕
│ NOME DO DATASET                     │ MODELO               │   ACURÁCIA │    F1 │   AUC │
╞═════════════════════════════════════╪══════════════════════╪════════════╪═══════╪═══════╡
│ IndianLiverPatientDataset.csv       │ KNeighborsClassifier │      0.65  │ 0.513 │ 0.603 │
├─────────────────────────────────────┼──────────────────────┼────────────┼───────┼───────┤
│ HabermansSurvivalData.csv           │ KNeighborsClassifier │      0.703 │ 0.506 │ 0.541 │
├─────────────────────────────────────┼──────────────────────┼────────────┼───────┼───────┤
│ GermanCreditRisk.csv                │ KNeighborsClassifier │      0.627 │ 0.484 │ 0.501 │
├─────────────────────────────────────┼──────────────────────┼────────────┼───────┼───────┤
│ PortugueseBankMarketing.csv         │ KNeighborsClassifier │      0.882 │ 0.642 │ 0.765 │
├─────────────────────────────────────┼──────────────────────┼────────────┼─────

In [17]:
# Processamento de cada dataset - GradientBoostingClassifier
resultados_dataset = []
res_metrica_tabela = []

modelo = GradientBoostingClassifier(n_estimators=20, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo)

  if dados is not None:
    X, y, X_onehot = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados_dataset = avaliar_dataset(y, predicao, probabilidade)
    resultados_dataset_formatados = formatar_dados(resultados_dataset)
    # print(f'\n  Resultados do dataset {arquivo}: {resultados_dataset_formatados}')

    res_metrica_tabela.append([
      arquivo,
      str(modelo).split('(')[0],
      resultados_dataset_formatados.get('accuracy'),
      resultados_dataset_formatados.get('f1_score'),
      resultados_dataset_formatados.get('roc_auc_score')
    ])

    auc_geral = resultados_dataset.get('roc_auc_score')

    """
    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)
    """

    df_segmentos = avaliar_segmentos(X, y, predicao, probabilidade, X_onehot, avaliar_dataset, auc_geral, arquivo, dicio_atributos_sensiveis.get(arquivo, {}), 5)

    nome_dataset = arquivo.replace('.csv', '')
    arquivo_segmento = f"Segmentos_{nome_dataset}_GB.csv"
    caminho_segmento = os.path.join(caminho_resultados + "GradientBoosting/", arquivo_segmento)
    df_segmentos.to_csv(caminho_segmento, index=False)
  else:
    print(f'Não foi possível processar o dataset {arquivo}')

  Segmento nan sem dados (sem One Hot)


In [18]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═════════════════════════════════════╤════════════════════════════╤════════════╤═══════╤═══════╕
│ NOME DO DATASET                     │ MODELO                     │   ACURÁCIA │    F1 │   AUC │
╞═════════════════════════════════════╪════════════════════════════╪════════════╪═══════╪═══════╡
│ IndianLiverPatientDataset.csv       │ GradientBoostingClassifier │      0.695 │ 0.491 │ 0.731 │
├─────────────────────────────────────┼────────────────────────────┼────────────┼───────┼───────┤
│ HabermansSurvivalData.csv           │ GradientBoostingClassifier │      0.765 │ 0.622 │ 0.686 │
├─────────────────────────────────────┼────────────────────────────┼────────────┼───────┼───────┤
│ GermanCreditRisk.csv                │ GradientBoostingClassifier │      0.744 │ 0.607 │ 0.754 │
├─────────────────────────────────────┼────────────────────────────┼────────────┼───────┼───────┤
│ PortugueseBankMarketing.csv         │ GradientBoostingClassifier │      0.898 │ 0.649 │ 0.897 │
├───────────────────